In [1]:
import pandas
import requests
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from pandas import DataFrame
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

records = []
links_reportagem = []

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)


driver = webdriver.Chrome('C:/chromedriver/chromedriver.exe')

def limpaArrays(array):
    igual = set()
    result = []
    for item in array:
        if item not in igual:
            igual.add(item)
            result.append(item)
    return result

def gera_json():
    records_limpos = limpaArrays(records)
    df = DataFrame(records_limpos, columns=['comentarios'])
    with open('pacote_birulilo_teste_uol.json', 'w', encoding='utf-8') as file:
        df.to_json(file, force_ascii=False, orient='index')
        
def pega_comentarios():
    res = driver.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(res, 'html.parser') 
    comentarios_ = soup.find_all("p",{"class":"comment-text ng-binding ng-scope"})
    for comentarios in comentarios_:
        resultado_comentario = comentarios.contents[0]
        resultado_comentario.strip()
        records.append((resultado_comentario))
    
            
def botao_mais_respostas():
    while True:
        try:
            mais_respostas = driver.find_element_by_xpath("//div/div/div/div[4]//div/button[@class='btn btn-secondary load-comment more-comments btn-medium']")
            time.sleep(3)
            ActionChains(driver).move_to_element(mais_respostas).click(mais_respostas).perform()
        except NoSuchElementException as e:
            break
        except StaleElementReferenceException:
            break
    

def botao_respostas():
    while True:
        try:
            mais_comentarios = driver.find_element_by_xpath("//div/div/div/div/div/div/div/div/footer/a/i[@class='icon-single-arrow-down']")
            time.sleep(3)
            if(mais_comentarios.is_displayed()):
                ActionChains(driver).move_to_element(mais_comentarios).click(mais_comentarios).perform()
                botao_mais_respostas()
        except NoSuchElementException as e:
            break  
    pega_comentarios()
    
def botao_carrega():
    while True:
        try:
            driver.implicitly_wait(10)
            carrega_mais = driver.find_element_by_xpath('//div/div/div/div[5]/button')
            ActionChains(driver).move_to_element(carrega_mais).click(carrega_mais).perform()
        except NoSuchElementException as e:
            break
        except StaleElementReferenceException:
            break
    botao_respostas()

def busca_reportagens(url):
    driver.get(url)
    driver.maximize_window()
    try:
        target = driver.find_element_by_class_name('slot-b')
        driver.execute_script('arguments[0].scrollIntoView(true);', target)
    except:
        return link
    finally:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="comments ng-scope"]'))
            )
        finally: 
            botao_carrega()
        
    

def __init__(url):
    busca_reportagens(url)
    
__init__('https://noticias.uol.com.br/internacional/ultimas-noticias/2019/09/23/nao-precisamos-de-oracao-precisamos-de-acao-diz-jovem-brasileira-na-onu.htm')
gera_json()